In [2]:
.libPaths("/home/mahat/.conda/envs/r422/lib/R/library")
# .libPaths(c("/home/mahat/.conda/envs/r422/lib/R/library",
#             "/net/bmc-lab2/data/lab/scgroseq/group/software/R/x86_64-pc-linux-gnu-library/4.2"))
.libPaths()

[1] "/net/bmc-lab2/data/lab/sharp/mahat/conda/envs/r422/lib/R/library"

In [6]:
suppressMessages({
    library(tidyverse)
    library(dplyr)
    library(matrixStats)
    library(foreach)
    library(doParallel)
    library(plyranges)
    library(viridis)
    library(ggpointdensity)
    library(Matrix)
    library(data.table)
    library(Hmisc)
    library(gplots)
    library(rstatix)
    library(scattermore)
    library(igraph)
    library(ggraph)
    library(clusterProfiler)
    library(rtracklayer)
    library(stringr)
    library(org.Mm.eg.db)
    library(circlize)
    library(gplots)
    library(RIdeogram)
    library(enrichplot)
    library(GOSemSim)
    library(dynamicTreeCut)
    library(WGCNA)
});
source("./scGRO_functions.r");

In [7]:
registerDoParallel(16);
options(
    repr.plot.width=8,
    repr.plot.height=6,
    jupyter.plot_mimetypes = "image/svg+xml",
    digits=5
);
theme_set(theme_classic() +
    theme(
        axis.title.x = element_text(color="black", size=14, face="bold"), 
        axis.title.y = element_text(color="black", size=14, face="bold"),
        axis.text = element_text(color="black", size=12, face="bold"),
        plot.title = element_text(face="bold", size=14, hjust = 0.5),
        axis.line = element_blank(),
        # axis.ticks = element_blank()
        panel.border = element_rect(colour = "grey", fill=NA, linewidth=1)
    )
);

In [8]:
# load genes with dREG status for expression filter
genesWithdREGstatus = readRDS("../data/groHMM_dREG_refinedGenes_mES_mm10.rds");
genesWithdREGstatus
# load groHMM-extended genes and enhancers
# features = read_bed("../data/groHMM_mES_BRsComb_LP-50_UTS10_features_customized_v2.bed");

# load features
features = read_bed("../data/groHMM_dREG_refinedFeatures_mES_mm10_OSNcustomEnhancers_SEs.bed");
# features = read_bed("../data/mES_BRsComb_dREGfiltered_features_customized_OSNenhancersPlusSEs_v1.bed");
# features = read_bed("../data/groHMM_mES_BRsComb_LP-50_UTS10_features_customized_OSNenhancersPlusSEs_v2.bed");
names(features) = features$name;
# features$name=NULL;
features$score=NULL;
features

GRanges object with 23980 ranges and 2 metadata columns:
                  seqnames            ranges strand |            name      dREG
                     <Rle>         <IRanges>  <Rle> |     <character> <logical>
        GN-Lypla1     chr1   4807559-4857592      + |       GN-Lypla1      TRUE
         GN-Tcea1     chr1   4857693-4916049      + |        GN-Tcea1      TRUE
       GN-Atp6v1h     chr1   5082977-5170849      + |      GN-Atp6v1h      TRUE
         GN-Oprk1     chr1   5588449-5606134      + |        GN-Oprk1      TRUE
        GN-Rb1cc1     chr1   6214661-6287049      + |       GN-Rb1cc1      TRUE
              ...      ...               ...    ... .             ...       ...
  GN-LOC108168665     chrY 86943635-86944319      - | GN-LOC108168665     FALSE
  GN-LOC108168654     chrY 88804769-88805453      - | GN-LOC108168654     FALSE
       GN-Gm21996     chrY 89700320-89745550      - |      GN-Gm21996     FALSE
  GN-LOC108168534     chrY 90737350-90750205      - | GN-LOC108

GRanges object with 36559 ranges and 1 metadata column:
                seqnames            ranges strand |          name
                   <Rle>         <IRanges>  <Rle> |   <character>
      GN-Lypla1     chr1   4807559-4857592      + |     GN-Lypla1
       GN-Tcea1     chr1   4857693-4916049      + |      GN-Tcea1
     GN-Atp6v1h     chr1   5082977-5170849      + |    GN-Atp6v1h
       GN-Oprk1     chr1   5588449-5606134      + |      GN-Oprk1
      GN-Rb1cc1     chr1   6214661-6287049      + |     GN-Rb1cc1
            ...      ...               ...    ... .           ...
  chrY-90724550     chrY 90724550-90727549      * | chrY-90724550
  chrY-90737450     chrY 90737450-90742449      * | chrY-90737450
  chrY-90742920     chrY 90742920-90745919      * | chrY-90742920
  chrY-90760850     chrY 90760850-90765849      * | chrY-90760850
  chrY-90774850     chrY 90774850-90777849      * | chrY-90774850
  -------
  seqinfo: 21 sequences from an unspecified genome; no seqlengths

In [9]:
# genes shorter than 1500 are: 2931
# genes shorter than 1000 are: 2209
# genes shorter than  500 are:  484
# genes shorter than  200 are:  147

# short Gm### genes
shortGm = features %>%
    filter( substr(names, 0, 3) == "GN-" ) %>%
    filter( width > 200 ) %>%
    filter( width < 1000 & substr(names, 0, 5) == "GN-Gm" );
length(shortGm)
summary(width(shortGm))
names(shortGm)[1:5]

# define highly expressed "blacklist" genes to be
# removed from correlations
blacklist = grep( "GN-Rp[ls]" , names(features) );
blacklist = grep( "GN-Sn[oh]", names(features) ) %>%
    append(blacklist);

blacklist = names(features)[ blacklist ];
length(blacklist)
summary(width(features[ blacklist ]))
blacklist[c(1:5, 110:115)]

[1] 614

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    203     401     626     612     815     999 

[1] "GN-Gm8210"  "GN-Gm33795" "GN-Gm10561" "GN-Gm20257" "GN-Gm39669"

[1] 117

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
    375    6785   10938   16845   15023  200344 

[1] "GN-Snhg6"   "GN-Snhg7os" "GN-Snhg11"  "GN-Snhg7"   "GN-Snhg17" 
 [6] "GN-Rps6kb2" "GN-Rps6ka4" "GN-Rpl10"   "GN-Rpl36a"  "GN-Rps6ka3"
[11] "GN-Rpl39"

In [10]:
# filter genes
genes = features %>%
    filter( substr(names, 0, 3) == "GN-" ) %>%
    # remove genes less than 1500 nt
    filter( width >= 1500 ) %>%
    # filter genes with dREG peaks
    filter( names %in% names(genesWithdREGstatus[genesWithdREGstatus$dREG == T]) | 
           !names %in% names(genesWithdREGstatus) ) %>%
    # filter highly expressed genes
    filter(!names %in% blacklist ) %>%
    # filter Gm genes shorter than 1 kb
    filter(!names %in% names(shortGm) ) %>%
    # remove 500 nt at the 5' end
    anchor_3p() %>%
    mutate( width = width - 500 );
summary(width(genes))
genes

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1000   12427   25879   54928   57279 2270160 

GRanges object with 15101 ranges and 1 metadata column:
                  seqnames              ranges strand |            name
                     <Rle>           <IRanges>  <Rle> |     <character>
        GN-Lypla1     chr1     4808059-4857592      + |       GN-Lypla1
         GN-Tcea1     chr1     4858193-4916049      + |        GN-Tcea1
       GN-Atp6v1h     chr1     5083477-5170849      + |      GN-Atp6v1h
         GN-Oprk1     chr1     5588949-5606134      + |        GN-Oprk1
        GN-Rb1cc1     chr1     6215161-6287049      + |       GN-Rb1cc1
              ...      ...                 ...    ... .             ...
        GN-Frmpd4     chrX 167471306-168576734      - |       GN-Frmpd4
          GN-Msl3     chrX 168646600-168673372      - |         GN-Msl3
          GN-Hccs     chrX 169305550-169319872      - |         GN-Hccs
          GN-Srsy     chrY   17935713-17936830      - |         GN-Srsy
  GN-LOC108168644     chrY   90822500-90828895      - | GN-LOC108168644
  ------

In [11]:
enhancers = features %>%
    filter( substr(names, 0, 3) != "GN-" ) %>%
    filter( width >= 3000 );
summary(width(enhancers))
table(substr(enhancers$name, 0, 3));
enhancers

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   3000    3000    3000    3394    3000   45151 


  chr   INT   Klf   Nan   Pou   Sox 
12294   204     1     7     9    10 

GRanges object with 12525 ranges and 1 metadata column:
                seqnames            ranges strand |          name
                   <Rle>         <IRanges>  <Rle> |   <character>
   chr1-3061160     chr1   3061160-3064159      * |  chr1-3061160
   chr1-3952295     chr1   3952295-3955294      * |  chr1-3952295
   chr1-4491400     chr1   4491400-4494399      * |  chr1-4491400
   chr1-4570350     chr1   4570350-4573349      * |  chr1-4570350
   chr1-4687525     chr1   4687525-4690524      * |  chr1-4687525
            ...      ...               ...    ... .           ...
  chrY-90724550     chrY 90724550-90727549      * | chrY-90724550
  chrY-90737450     chrY 90737450-90742449      * | chrY-90737450
  chrY-90742920     chrY 90742920-90745919      * | chrY-90742920
  chrY-90760850     chrY 90760850-90765849      * | chrY-90760850
  chrY-90774850     chrY 90774850-90777849      * | chrY-90774850
  -------
  seqinfo: 21 sequences from an unspecified genome; no seqlengths

In [12]:
# load groHMM-extended genes and enhancers
dREG = read.table("../data/PROseq_mES_BRsComb.dREG.peak.full.bed", header=F, stringsAsFactors=F);
colnames(dREG) = c("chr", "start", "end", "score", "pval", "center");
# use the "center" column as start
# colnames(dREG) = c("chr", "leftEnd", "rightEnd", "score", "pval", "start");
# dREG$end = dREG$start;
dREG = GRanges(dREG);
summary(dREG$score)
 # dREG = dREG %>%
 #    # removing dREG peaks with score lower than mean
 #    filter( score > median(score) );

# mark dREG peaks by the overlapping enhancers (from freatures):
hits = findOverlaps( dREG, enhancers, ignore.strand=T );
dREG$assignedEnhancer = NA;
# assign ovelapping SE to dREG peaks
dREG$assignedEnhancer[hits@from] = names(enhancers)[hits@to];

# filter dREG peak to only leave one with highest score:
dREGfilt = dREG %>%
    # filter dREG peaks that do not overlap with an enhancer
    na.omit() %>%
    group_by( assignedEnhancer ) %>%
    filter( score == max(score) ) %>%
    ungroup() %>%
    # make data table for left_join
    as.data.table;
dim(dREGfilt)
table(substr(dREGfilt$assignedEnhancer, 0, 3));
length(unique(dREGfilt$assignedEnhancer))
dREGfilt[1:4, ]

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.282   0.484   0.672   0.722   0.939   1.573 

[1] 12400     9


  chr   INT   Klf   Pou   Sox 
12262   131     1     2     3 

[1] 12400

seqnames,start,end,width,strand,score,pval,center,assignedEnhancer
<fct>,<int>,<int>,<int>,<fct>,<dbl>,<dbl>,<int>,<chr>
chr1,3062460,3062860,401,*,0.31873,0.04351726,3062700,chr1-3061160
chr1,3953590,3954000,411,*,0.62957,0.00081164,3953790,chr1-3952295
chr1,4493370,4493820,451,*,0.62843,0.00022320,4493640,chr1-4491400
chr1,4571180,4572240,1061,*,1.13141,0.00000000,4571860,chr1-4570350


In [13]:
Ecenter = left_join( as.data.table(enhancers), dREGfilt[,c(6:9)], by=c("name"="assignedEnhancer") ) %>%
    na.omit() #%>%
    # GRanges()
# names(Ecenter) = Ecenter$name;
Ecenter$pval = NULL;
Ecenter$score = NULL;
Ecenter[1:4, ]

seqnames,start,end,width,strand,name,center
<fct>,<int>,<int>,<int>,<fct>,<chr>,<int>
chr1,3061160,3064159,3000,*,chr1-3061160,3062700
chr1,3952295,3955294,3000,*,chr1-3952295,3953790
chr1,4491400,4494399,3000,*,chr1-4491400,4493640
chr1,4570350,4573349,3000,*,chr1-4570350,4571860


In [14]:
# ## join SE and dREG with dREG center:
# # convert center to start and end
# Ecenter = left_join( as.data.table(enhancers)[,c(1,6)], dREGfilt[,c(6:9)], by=c("name"="assignedEnhancer") ) %>%
#       # some NAs come from missing entries
#       na.omit() %>%
#       mutate( start = center ) %>%
#       mutate( end = center ) %>%
#       GRanges()# %>%
# names(Ecenter) = Ecenter$name;
# # SEcenter$name = NULL;
# Ecenter$center = NULL;

# # create 1500 nt bin of Enhancer in sense direction
# Edown = resize(Ecenter, width = 1500, fix="start") %>%
#     mutate( strand = "+" ) %>%
#     subsetByOverlaps( genes, invert = TRUE );
# length(Edown)
# # create 1500 nt bin of Enhancer in anti-sense direction
# Eup = resize(Ecenter, width = 1500, fix="end") %>%
#     mutate( strand = "-" ) %>%
#     subsetByOverlaps( genes, invert = TRUE );
# length(Eup)

In [15]:
# create Enhancer bin in sense direction
Edown = Ecenter %>%
    mutate( strand = "+" ) %>%
    #convert center to start
    mutate( start = center ) %>%
    # make sure the end is at least 1750 For > 1750, keep as it is
    mutate( end = ifelse(end - center > 1750, end, center + 1750) ) %>%
    GRanges() %>%
    anchor_3p() %>%
    mutate( width = width - 250 )
    # # no need for this beacuse the genes will be filtered against enhancers:
    # subsetByOverlaps( genes, invert = TRUE );
names(Edown) = Edown$name;
Edown$center = NULL;
Edown
# create Enhancer bin in anti-sense direction
Eup = Ecenter %>%
    mutate( strand = "-" ) %>%
    mutate( end = center) %>%
    mutate( start = ifelse(center - start > 1750, start, center - 1750) ) %>%
    GRanges() %>%
    anchor_3p() %>%
    mutate( width = width - 250 );
names(Eup) = Eup$name;
Eup$center = NULL;
Eup

GRanges object with 12399 ranges and 1 metadata column:
                seqnames            ranges strand |          name
                   <Rle>         <IRanges>  <Rle> |   <character>
   chr1-3061160     chr1   3062950-3064450      + |  chr1-3061160
   chr1-3952295     chr1   3954040-3955540      + |  chr1-3952295
   chr1-4491400     chr1   4493890-4495390      + |  chr1-4491400
   chr1-4570350     chr1   4572110-4573610      + |  chr1-4570350
   chr1-4687525     chr1   4689370-4690870      + |  chr1-4687525
            ...      ...               ...    ... .           ...
  chrY-90724550     chrY 90726450-90727950      + | chrY-90724550
  chrY-90737450     chrY 90742010-90743510      + | chrY-90737450
  chrY-90742920     chrY 90744770-90746270      + | chrY-90742920
  chrY-90760850     chrY 90764140-90765849      + | chrY-90760850
  chrY-90774850     chrY 90777440-90778940      + | chrY-90774850
  -------
  seqinfo: 21 sequences from an unspecified genome; no seqlengths

GRanges object with 12399 ranges and 1 metadata column:
                seqnames            ranges strand |          name
                   <Rle>         <IRanges>  <Rle> |   <character>
   chr1-3061160     chr1   3060950-3062450      - |  chr1-3061160
   chr1-3952295     chr1   3952040-3953540      - |  chr1-3952295
   chr1-4491400     chr1   4491400-4493390      - |  chr1-4491400
   chr1-4570350     chr1   4570110-4571610      - |  chr1-4570350
   chr1-4687525     chr1   4687370-4688870      - |  chr1-4687525
            ...      ...               ...    ... .           ...
  chrY-90724550     chrY 90724450-90725950      - | chrY-90724550
  chrY-90737450     chrY 90737450-90741510      - | chrY-90737450
  chrY-90742920     chrY 90742770-90744270      - | chrY-90742920
  chrY-90760850     chrY 90760850-90763640      - | chrY-90760850
  chrY-90774850     chrY 90774850-90776940      - | chrY-90774850
  -------
  seqinfo: 21 sequences from an unspecified genome; no seqlengths

In [16]:
# combine features:
expressedFeatures = c(Eup, Edown, genes) %>%
     sort();
summary(width(expressedFeatures))
expressedFeatures

# trim the long genes to 10 kb
longf = which( width(expressedFeatures) >= 10000 );
expressedFeatures[longf] = expressedFeatures[longf] %>%
    resize( width = 10000, fix="start" );
summary(width(expressedFeatures))
expressedFeatures

# writing the file to check in IGV:
# write_bed(expressedFeatures, file="../data/ExpressedFeatures_4c_ExG_All_Corr.bed")

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1000    1501    1501   21845   16584 2270160 

GRanges object with 39899 ranges and 1 metadata column:
                  seqnames            ranges strand |            name
                     <Rle>         <IRanges>  <Rle> |     <character>
     chr1-3061160     chr1   3062950-3064450      + |    chr1-3061160
     chr1-3952295     chr1   3954040-3955540      + |    chr1-3952295
     chr1-4491400     chr1   4493890-4495390      + |    chr1-4491400
     chr1-4570350     chr1   4572110-4573610      + |    chr1-4570350
     chr1-4687525     chr1   4689370-4690870      + |    chr1-4687525
              ...      ...               ...    ... .             ...
    chrY-90737450     chrY 90737450-90741510      - |   chrY-90737450
    chrY-90742920     chrY 90742770-90744270      - |   chrY-90742920
    chrY-90760850     chrY 90760850-90763640      - |   chrY-90760850
    chrY-90774850     chrY 90774850-90776940      - |   chrY-90774850
  GN-LOC108168644     chrY 90822500-90828895      - | GN-LOC108168644
  -------
  seqinfo: 21 sequences 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1000    1501    1501    4510   10000   10000 

GRanges object with 39899 ranges and 1 metadata column:
                  seqnames            ranges strand |            name
                     <Rle>         <IRanges>  <Rle> |     <character>
     chr1-3061160     chr1   3062950-3064450      + |    chr1-3061160
     chr1-3952295     chr1   3954040-3955540      + |    chr1-3952295
     chr1-4491400     chr1   4493890-4495390      + |    chr1-4491400
     chr1-4570350     chr1   4572110-4573610      + |    chr1-4570350
     chr1-4687525     chr1   4689370-4690870      + |    chr1-4687525
              ...      ...               ...    ... .             ...
    chrY-90737450     chrY 90737450-90741510      - |   chrY-90737450
    chrY-90742920     chrY 90742770-90744270      - |   chrY-90742920
    chrY-90760850     chrY 90760850-90763640      - |   chrY-90760850
    chrY-90774850     chrY 90774850-90776940      - |   chrY-90774850
  GN-LOC108168644     chrY 90822500-90828895      - | GN-LOC108168644
  -------
  seqinfo: 21 sequences 

In [17]:
# filter out features that overlap:
hits = findOverlaps(expressedFeatures)
expressedFeatures$overlappingFeature = NA;
# assign ovelapping feature
expressedFeatures$overlappingFeature[hits@from] = names(expressedFeatures)[hits@to];

test = expressedFeatures %>%
    na.omit() %>%
    group_by( overlappingFeature ) %>%
    filter( n_distinct(name) == 1 ) %>%
    ungroup();
test

GRanges object with 39478 ranges and 2 metadata columns:
                  seqnames            ranges strand |            name
                     <Rle>         <IRanges>  <Rle> |     <character>
     chr1-3061160     chr1   3062950-3064450      + |    chr1-3061160
     chr1-3952295     chr1   3954040-3955540      + |    chr1-3952295
     chr1-4491400     chr1   4493890-4495390      + |    chr1-4491400
     chr1-4570350     chr1   4572110-4573610      + |    chr1-4570350
     chr1-4687525     chr1   4689370-4690870      + |    chr1-4687525
              ...      ...               ...    ... .             ...
    chrY-90737450     chrY 90737450-90741510      - |   chrY-90737450
    chrY-90742920     chrY 90742770-90744270      - |   chrY-90742920
    chrY-90760850     chrY 90760850-90763640      - |   chrY-90760850
    chrY-90774850     chrY 90774850-90776940      - |   chrY-90774850
  GN-LOC108168644     chrY 90822500-90828895      - | GN-LOC108168644
                  overlappingFeat

In [18]:
scGRO = readRDS("../data/scGROv2p8_consolidated.rds");
counts = readRDS("../data/scGROv2p8_mapq3qc_max10kbp_filtered_counts.rds");
#counts = counts[,colSums(counts) >= 1000 ];
dim(counts)
allcells = colnames(counts);
table(substr(allcells, 0, 6));

[1] 36559  2635


Exp156 Exp160 Exp208 Exp211 Exp236 Exp246 Exp260 Exp263 Exp264 
   104    100    142    115    835     93    679    268    299 

In [19]:
# Merge experiment ID and cell barcode to create unique cell ID across experiments
scGRO = scGRO %>%
    filter( mapq >= 3 & countQC & umiQC & plateQC & cellQC & miRQC ) %>%
    subsetByOverlaps( expressedFeatures ) %>%
    mutate( cellID = factor( paste( Exp, Plate, Cell, sep="-") ) ) %>%
    filter( cellID %in% allcells ) #%>%
    # select( cellID );
scGRO

GRanges object with 1217969 ranges and 12 metadata columns:
            seqnames            ranges strand |      mapq    PolPos      Exp
               <Rle>         <IRanges>  <Rle> | <integer> <integer> <factor>
        [1]     chr1   3064450-3064492      + |        22   3064492  Exp236 
        [2]     chr1   3064450-3064492      + |        22   3064492  Exp236 
        [3]     chr1   4494883-4494900      + |         7   4494900  Exp236 
        [4]     chr1   4573099-4573116      + |         7   4573116  Exp236 
        [5]     chr1   4690719-4690736      + |         7   4690736  Exp260b
        ...      ...               ...    ... .       ...       ...      ...
  [1217965]     chrY 90828680-90828724      - |        22  90828680  Exp236 
  [1217966]     chrY 90828694-90828728      - |        42  90828694  Exp260b
  [1217967]     chrY 90828695-90828735      - |        28  90828695  Exp236 
  [1217968]     chrY 90828695-90828737      - |         4  90828695  Exp236 
  [1217969]     

In [20]:
counts = foreach(
    id = allcells,
    .combine="cbind2"
) %dopar% {
    reads = scGRO %>% filter(cellID == id);
    # count reads in each feature for this cellID
    counts1 = countOverlaps( expressedFeatures, reads );

    counts = cbind(counts1) %>%
        Matrix(sparse=T);

    return( counts );
}
# columns = cells = cellIDs
colnames(counts) = allcells;
rownames(counts) = names(expressedFeatures);
dim(counts)

In [ ]:
# to sum rows with same rownames
# the sense and anti-sense bins of enhancers have same names
counts = rowsum(counts, row.names(counts))
dim(counts)
# rowSums(counts)
# colSums(counts)

In [ ]:
data.frame(
    x=rowMeans(counts>0)
) %>%
    ggplot( aes(x=x) ) +
    geom_histogram(binwidth=0.05) +
    scale_x_log10() +
    ggtitle("Reads per Feature") +
    xlab("Reads per cell") +
    ylab("Number of features");

In [ ]:
# only retain features with counts in 0.1% or more cells
observed = counts[ rowMeans(counts>0) >= 0.001, ];
dim(observed)

test = substr(rownames(observed), 0, 3)
summary(as.factor(test))

## ExG Correlation & p-value from chi-square test:

In [20]:
# The matrix had to be converted to integer type for chi-square function written by chatGPT:
forceMatrixToInteger <- function(m){
    apply (m, c (1, 2), function (x) {
         (as.integer(x))
    })
}

In [21]:
# transforming the matrix to calculate corr, hmisc package reqires genes in columns and cells in rows

obsx = as.matrix(t(observed))
# obsx = as.matrix(t(observed[1:100, 1:50]))
# Binarize observed counts
# obsx = as.matrix(t(observed[1:100, 1:50]))
obsx[obsx > 1] = 1;

obsB = forceMatrixToInteger(obsx)
dim(obsB)

[1]  2635 23538

In [22]:
# Function to calculate chi-square p-value for a pair of genes
chi_square_test = function(gene1, gene2) {
  # Create contingency table
  table = table(gene1, gene2)
  
  # Perform chi-square test
  test = chisq.test(table, correct = FALSE) # 'correct = FALSE' to avoid Yates' correction
  return(test$p.value)
}

In [23]:
# ++++++++++++++++++++++++++++
# flattenCorrMatrix
# ++++++++++++++++++++++++++++
# cormat : matrix of the correlation coefficients
# pmat : matrix of the correlation p-values
flattenCorrMatrix = function(cormat, pmat) {
  ut = upper.tri(cormat)
  data.frame(
    geneA = rownames(cormat)[row(cormat)[ut]],
    geneB = rownames(cormat)[col(cormat)[ut]],
    corr = (cormat)[ut],
    pVal = pmat[ut]
    )
}

In [ ]:
# Apply the function to all pairs of genes
suppressWarnings({
    chSq_pVal = outer(1:ncol(obsB), 1:ncol(obsB), Vectorize(function(i, j) chi_square_test(obsB[,i], obsB[,j])))
                })
# colnames(p_values) = colnames(data)
# rownames(p_values) = colnames(data);
dim(chSq_pVal)
chSq_pVal[1:5,1:5]                                           

In [ ]:
# use hmisc package to calculate correlation and p-value:
corrP = rcorr(obsB, type = "pearson")
dim(corrP$r)
corrP$r[1:5,1:5]
dim(corrP$P)
corrP$P[1:5,1:5]

In [ ]:
corrBflat = flattenCorrMatrix( corrP$r, chSq_pVal ) %>%
    mutate( pAdj = p.adjust(pVal, method = "bonferroni") ) %>%
    mutate( corrRank = rank(corr) ) %>%
    mutate( pValRank = rank(pVal) );
dim(corrBflat)
head(corrBflat)
fwrite(corrBflat, file="../data/scGROv2p8_ExG_max10kbp_binary_correlation_chiSquare.csv.gz")